In [ ]:
import pandas as pd
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import logging
from mylib.lib import data_split_to_feature_outcome, model_execution, rank_teams_produce_top_68, postseason_result, post_season_mapping

In [2]:
logging.basicConfig(
    filename='data_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

In [3]:
logging.info("Loading training and testing datasets.")
cbb = pd.read_csv('../Final_Project_DE/train_data.csv')
cbb24 = pd.read_csv('../Final_Project_DE/test_data.csv')

In [4]:
features = [
    'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR', 'TORD', 'ORB',
    'DRB', 'FTR', 'FTRD', 'ADJ_T', 'WAB'
]
outcome = 'SEED'
X,y = data_split_to_feature_outcome(cbb, outcome, features)

# Extract the features from cbb24 for prediction
cbb24_seedless = cbb24[features]

cbb24['predicted_seed_score'] = model_execution(cbb24_seedless, X, y)

cbb24 = rank_teams_produce_top_68(cbb24, 'predicted_seed_score')


/Users/visheshgupta/Desktop/Final_Project_DE/mylib/lib.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model[outcome] = df_model[outcome].astype(int)
  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:05
 18% (2 of 11) |####                     | Elapsed Time: 0:00:01 ETA:   0:00:04
 27% (3 of 11) |######                   | Elapsed Time: 0:00:01 ETA:   0:00:04
 36% (4 of 11) |#########                | Elapsed Time: 0:00:02 ETA:   0:00:04
 45% (5 of 11) |###########              | Elapsed Time: 0:00:03 ETA:   0:00:04
 54% (6 of 11) |#############            | Elapsed Time: 0:00:03 ETA:   0:00:03
 63% (7 of 11) |###############       

Mean Absolute Error (MAE): 1.088235294117647


In [6]:
current_data = pd.read_csv('../Final_Project_DE/current_cbb_live_data.csv')

logging.info("Execute model on our current dataset i.e. current_cbb_live_data to get the seeds for 2024-2025 season")

current_data_seedless = current_data[features]

current_data['predicted_seed_score'] = model_execution(current_data_seedless, X, y)

current_data = rank_teams_produce_top_68(current_data, 'predicted_seed_score')

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--
  9% (1 of 11) |##                       | Elapsed Time: 0:00:00 ETA:   0:00:06
 18% (2 of 11) |####                     | Elapsed Time: 0:00:01 ETA:   0:00:06
 27% (3 of 11) |######                   | Elapsed Time: 0:00:02 ETA:   0:00:05
 36% (4 of 11) |#########                | Elapsed Time: 0:00:02 ETA:   0:00:04
 45% (5 of 11) |###########              | Elapsed Time: 0:00:02 ETA:   0:00:03
 54% (6 of 11) |#############            | Elapsed Time: 0:00:03 ETA:   0:00:02
 63% (7 of 11) |###############          | Elapsed Time: 0:00:03 ETA:   0:00:02
 72% (8 of 11) |##################       | Elapsed Time: 0:00:04 ETA:   0:00:01
 81% (9 of 11) |####################     | Elapsed Time: 0:00:04 ETA:   0:00:01
 90% (10 of 11) |#####################   | Elapsed Time: 0:00:05 ETA:   0:00:00
100% (11 of 11) |########################| Elapsed Time: 0:00:06 Time:  0:00:06


Mean Absolute Error (MAE): 1.088235294117647


In [7]:
current_date = datetime.now().strftime("%Y%m%d")
file_name = f"seeding_2025_{current_date}.csv"
current_data[['TEAM', 'predicted_seed_with_update']].sort_values('predicted_seed_with_update').dropna(subset=['predicted_seed_with_update']).to_csv(file_name)

In [8]:
# Preprocess the POSTSEASON column in cbb
cbb = cbb[cbb['POSTSEASON'].notnull()]  # Filter rows with POSTSEASON values
label_encoder = LabelEncoder()
cbb['POSTSEASON_LABEL'] = label_encoder.fit_transform(cbb['POSTSEASON'])  # Encode POSTSEASON

# Select features for modeling
postseason_features = [
    'ADJOE', 'ADJDE', 'BARTHAG', 'EFG_O', 'EFG_D', 'TOR',
    'TORD', 'ORB', 'DRB', 'FTR', 'FTRD',
    'ADJ_T', 'WAB'
]

logging.info("Run model to obtain predictions on who will be winning march madness for the 2024-25 season")

# Define X and y for training
X_postseason = cbb[postseason_features]
y_postseason = cbb['POSTSEASON_LABEL']

current_data_seeded = current_data.dropna(subset=['predicted_seed_with_update'])
current_data_seeded['predicted_postseason_label'] = postseason_result(X_postseason, y_postseason, label_encoder, current_data, 'predicted_seed_with_update', postseason_features)

              precision    recall  f1-score   support

         2ND       1.00      0.33      0.50         3
   Champions       0.00      0.00      0.00         0
          E8       0.25      0.25      0.25         8
          F4       0.00      0.00      0.00         7
         R32       0.36      0.31      0.33        32
         R64       0.60      0.85      0.71        61
         R68       0.50      0.11      0.18         9
         S16       0.10      0.06      0.08        16

    accuracy                           0.49       136
   macro avg       0.35      0.24      0.26       136
weighted avg       0.44      0.49      0.44       136



/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/miniconda3/lib/python3.12/site-packag

In [9]:
current_data_seeded['predicted_postseason_description'] = post_season_mapping(current_data_seeded, 'predicted_postseason_label')
current_data_seeded.sort_values('predicted_postseason_label').head(16)

/var/folders/mc/2wjfdchj6vsffbrpfbfgqw4w0000gn/T/ipykernel_60101/687930863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_data_seeded['predicted_postseason_description'] = post_season_mapping(current_data_seeded, 'predicted_postseason_label')


,TEAM,CONF,G,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,2P_D,3P_O,3P_D,ADJ_T,WAB,predicted_seed_score,rank,predicted_seed_with_update,predicted_postseason_label,predicted_postseason_description
4,Gonzaga,WCC,8,123.520,94.915,0.953650,55.4510,45.1370,12.9400,18.8119,...,50.7177,35.4990,24.5300,70.2740,1.8110,7.462990,7.0,2.0,0,Winner
3,Auburn,SEC,8,126.310,96.327,0.957740,59.7500,46.6630,13.8220,14.9304,...,44.1320,36.7690,35.0248,68.1182,3.0200,6.007616,1.0,1.0,1,Runner-Up
0,Duke,ACC,8,119.660,89.410,0.966110,55.6450,42.4140,15.4670,20.3740,...,41.5900,36.6730,29.2580,67.3227,1.3190,6.900835,4.0,1.0,2,Final Four
1,Houston,B12,7,119.211,90.520,0.959520,51.6143,44.4280,13.3900,22.6210,...,42.8200,42.5400,30.8101,63.5359,-0.4185,8.301586,14.0,4.0,2,Final Four
21,Michigan,B10,8,114.939,95.922,0.888622,57.1260,41.6600,21.2333,23.3120,...,41.0700,36.7690,28.4400,69.0128,1.1240,8.916438,17.0,5.0,3,Elite Eight
30,Creighton,BE,9,114.143,96.629,0.871531,54.2760,43.4200,19.0249,12.3353,...,42.0120,32.5198,30.6950,69.4107,-0.3179,9.638109,26.0,7.0,3,Elite Eight
28,Georgia,SEC,9,113.846,96.124,0.875429,56.4340,44.4280,19.8279,18.0152,...,46.4670,34.0141,27.5250,67.5215,1.2200,9.999993,35.0,9.0,3,Elite Eight
19,Cincinnati,B12,7,115.433,95.719,0.895920,59.2800,40.6300,13.5160,17.4190,...,40.4500,39.9150,27.4240,65.8295,0.3849,9.829786,31.0,8.0,3,Elite Eight
23,Arizona,B12,7,115.334,96.732,0.883424,51.4153,48.8123,16.3109,19.4980,...,44.9440,31.0252,35.6277,72.0220,-1.3140,10.435610,45.0,11.0,4,Sweet Sixteen
12,Mississippi St.,SEC,8,118.615,96.833,0.911513,53.4940,48.1105,12.8300,21.4510,...,45.3500,33.2170,34.8240,69.4105,1.4140,8.454153,15.0,4.0,4,Sweet Sixteen


In [10]:
current_date = datetime.now().strftime("%Y%m%d")
file_name = f"cbb25_seeded_{current_date}.csv"
current_data_seeded.to_csv(file_name, index=False)